## Importando bibliotecas

In [1]:
import pandas as pd  # Importa a biblioteca pandas, amplamente utilizada para análise e manipulação de dados em Python.
import sqlite3  # Importa o módulo sqlite3, que fornece uma interface SQL para bancos de dados SQLite.
from itertools import combinations  # Importa a função 'combinations' do módulo 'itertools', útil para gerar combinações de elementos.
import csv  # Importa o módulo 'csv', que permite ler e escrever arquivos CSV (Comma Separated Values).

Estabelecendo conexão no banco de dados

In [2]:
con_rais = sqlite3.connect("/content/drive/MyDrive/rais_arquivos2.db")

# Estabelece uma conexão com o banco de dados SQLite chamado 'rais_arquivos2.db' localizado no caminho especificado.
# A variável 'con_rais' agora representa essa conexão e pode ser usada para interagir com o banco de dados.

Criando função de contagem

In [ ]:
def contar_por_grupo(conexao, tabela, variaveis):
    # Esta função conta ocorrências únicas de combinações de variáveis em uma tabela de banco de dados.

    # Converte a lista de variáveis em uma string formatada para uso em consultas SQL.
    # Ex: ['UF', 'Sexo'] vira "UF, Sexo".
    colunas = ", ".join(variaveis)

    # Constrói a consulta SQL para selecionar as variáveis e contar as ocorrências.
    # A consulta agrupa os resultados pelas variáveis para obter a contagem por cada combinação única.
    sql = f"""
        SELECT {colunas}, COUNT(*) as estoque
        FROM {tabela}
        GROUP BY {colunas}
    """

    # Executa a consulta SQL no banco de dados usando a conexão fornecida
    # e retorna o resultado diretamente como um DataFrame do pandas.
    return pd.read_sql(sql, conexao)

In [ ]:
def contagem_total_estoque(conexao):
    # Esta função conta o número total de registros na tabela 'rais2023'.

    # Executa uma consulta SQL para obter a contagem total de linhas na tabela 'rais2023'.
    # O resultado é lido diretamente em um DataFrame do pandas.
    contagem = pd.read_sql(f"SELECT count(*) FROM rais2023;", conexao)

    # Renomeia a coluna resultante da contagem de 'count(*)' para 'estoque' para maior clareza.
    contagem = contagem.rename(columns={"count(*)": 'estoque'})

    # Extrai o valor numérico da contagem total, que está na primeira linha e primeira coluna do DataFrame.
    valor = contagem.iloc[0, 0]

    # Cria um novo DataFrame do pandas com colunas de categorias vazias
    # e preenche a coluna 'estoque' com o valor da contagem total.
    # O objetivo é padronizar a saída com a mesma estrutura de outros DataFrames de contagem.
    dados = pd.DataFrame([{
        'sigla_uf': "",
        'sexo': "",
        'raca_cor': "",
        'faixa_etaria': "",
        'grau_instrucao_apos_2005': "",
        'cnae_2_subclasse_descricao_secao': "",
        "estoque": valor
    }])

    # Retorna o DataFrame contendo a contagem total.
    return dados

## Realizando a criação da tabela de contagem

In [ ]:
# Lista de variáveis para agrupamento de dados.
variaveis = [
    'sigla_uf',                                    # Sigla da Unidade Federativa.
    'sexo',                                        # Sexo da pessoa.
    'raca_cor',                                    # Raça ou cor.
    'faixa_etaria',                                # Faixa etária.
    'grau_instrucao_apos_2005',                    # Grau de instrução (pós-2005).
    'cnae_2_subclasse_descricao_secao'             # Seção da Classificação Nacional de Atividades Econômicas.
]

# Essa lista define as **colunas-chave** que serão usadas para **agrupar** os dados.
# Ao passá-la para funções como `contar_por_grupo`, você obtém a contagem de registros
# para cada **combinação única** dessas categorias.

In [ ]:
def lista_de_combinacoes(n):
    # Inicializa uma lista vazia para armazenar todas as combinações que serão geradas.
    todas_combinacoes = []

    # Itera sobre os possíveis tamanhos de combinações (r), começando de 1 até o número total de variáveis (colunas).
    for r in range(1, len(variaveis) + 1):
        # Verifica se o tamanho da combinação atual (r) é igual ao valor 'n' passado como argumento para a função.
        # Isso significa que queremos combinações de um tamanho específico 'n'.
        if r == n:
            # Se o tamanho for 'n', gera as combinações das 'variaveis' (colunas) com tamanho 'r' e as adiciona à lista 'todas_combinacoes'.
            # A função 'combinations' retorna um iterador de tuplas, por isso usamos 'extend'.
            todas_combinacoes.extend(combinations(variaveis, r))
        else:
            # Se o tamanho da combinação 'r' não for igual a 'n', simplesmente ignora e passa para a próxima iteração.
            pass

    # Converte cada tupla de combinação em uma lista. Isso é útil para manipulações posteriores, se necessário.
    todas_combinacoes = [list(comb) for comb in todas_combinacoes]

    # Retorna a lista final contendo todas as combinações de colunas com o tamanho 'n' especificado.
    return todas_combinacoes

## Iniciando A criação da planilha

Primeira linha

In [ ]:
DADOS = contagem_total_estoque(con_rais)
DADOS

,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467


Contagem considerando uma variável

In [ ]:
# Processamento de Combinações e Concatenação de Dados (col_v1)

# Gera uma lista de combinações para agrupamento de dados.
# O argumento '1' provavelmente indica o nível mais básico ou singular de combinações.
col_v1 = lista_de_combinacoes(1)

# Itera sobre cada combinação na lista 'col_v1'.
for iter1 in col_v1:
    # Chama a função 'contar_por_grupo' para processar os dados
    # usando a conexão 'con_rais', a fonte 'rais2023' e a combinação atual 'iter1'.
    dados_v1 = contar_por_grupo(con_rais,'rais2023',iter1)

    # Concatena os resultados 'dados_v1' ao DataFrame principal 'DADOS'.
    # `pd.concat` adiciona as novas linhas ao final de 'DADOS'.
    # `ignore_index=True` garante que o índice do DataFrame resultante seja contínuo.
    DADOS = pd.concat([DADOS,dados_v1],ignore_index=True)

    # Imprime a combinação atual ('iter1') para acompanhar o progresso.
    print(iter1)

# Exibe o DataFrame 'DADOS' final, que agora contém os resultados de todas as contagens
# para as combinações geradas em 'col_v1'.
DADOS

['sigla_uf']
['sexo']
['raca_cor']
['faixa_etaria']
['grau_instrucao_apos_2005']
['cnae_2_subclasse_descricao_secao']


,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
71,NaN,NaN,NaN,NaN,NaN,Outras Atividades De Serviços,1667251
72,NaN,NaN,NaN,NaN,NaN,Saúde Humana E Serviços Sociais,4325905
73,NaN,NaN,NaN,NaN,NaN,Serviços Domésticos,1799
74,NaN,NaN,NaN,NaN,NaN,"Transporte, Armazenagem E Correio",3946129


In [ ]:
# Processamento de Combinações e Concatenação de Dados (col_v2)

# Gera uma lista de combinações para agrupamento de dados.
# O argumento '2' define o nível ou tipo de combinações que serão geradas.
col_v2 = lista_de_combinacoes(2)

# Itera sobre cada combinação na lista 'col_v2'.
for iter2 in col_v2:
    # Chama a função 'contar_por_grupo' para processar os dados
    # utilizando a conexão 'con_rais', a fonte 'rais2023' e a combinação atual 'iter2'.
    dados_v2 = contar_por_grupo(con_rais,'rais2023',iter2)

    # Concatena os resultados 'dados_v2' ao DataFrame principal 'DADOS'.
    # `pd.concat` adiciona as novas linhas ao final de 'DADOS'.
    # `ignore_index=True` garante que o índice do DataFrame resultante seja contínuo,
    # mesmo após a adição de novos dados.
    DADOS = pd.concat([DADOS,dados_v2],ignore_index=True)

    # Imprime a combinação atual ('iter2') para acompanhar o progresso da execução.
    print(iter2)

# Exibe o DataFrame 'DADOS' final, que agora inclui os resultados de todas as contagens
# para as combinações geradas em 'col_v2'.
DADOS

['sigla_uf', 'sexo']
['sigla_uf', 'raca_cor']
['sigla_uf', 'faixa_etaria']
['sigla_uf', 'grau_instrucao_apos_2005']
['sigla_uf', 'cnae_2_subclasse_descricao_secao']
['sexo', 'raca_cor']
['sexo', 'faixa_etaria']
['sexo', 'grau_instrucao_apos_2005']
['sexo', 'cnae_2_subclasse_descricao_secao']
['raca_cor', 'faixa_etaria']
['raca_cor', 'grau_instrucao_apos_2005']
['raca_cor', 'cnae_2_subclasse_descricao_secao']
['faixa_etaria', 'grau_instrucao_apos_2005']
['faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']


,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
2124,NaN,NaN,NaN,NaN,SUP. INCOMP,Outras Atividades De Serviços,79806
2125,NaN,NaN,NaN,NaN,SUP. INCOMP,Saúde Humana E Serviços Sociais,180671
2126,NaN,NaN,NaN,NaN,SUP. INCOMP,Serviços Domésticos,28
2127,NaN,NaN,NaN,NaN,SUP. INCOMP,"Transporte, Armazenagem E Correio",134183


In [ ]:
# Processamento de Combinações e Concatenação de Dados (col_v3)

# Gera uma lista de combinações para agrupamento de dados.
# O argumento '3' especifica o nível ou tipo de combinações a serem geradas.
col_v3 = lista_de_combinacoes(3)

# Itera sobre cada combinação na lista 'col_v3'.
for iter3 in col_v3:
    # Chama a função 'contar_por_grupo' para processar os dados
    # usando a conexão 'con_rais', a fonte 'rais2023' e a combinação atual 'iter3'.
    dados_v3 = contar_por_grupo(con_rais,'rais2023',iter3)

    # Concatena os resultados 'dados_v3' ao DataFrame principal 'DADOS'.
    # `pd.concat` adiciona as novas linhas ao final de 'DADOS'.
    # `ignore_index=True` garante que o índice do DataFrame resultante seja contínuo.
    DADOS = pd.concat([DADOS,dados_v3],ignore_index=True)

    # Imprime a combinação atual ('iter3') para monitorar o progresso do loop.
    print(iter3)

# Exibe o DataFrame 'DADOS' final, que agora contém os resultados de todas as contagens
# para as combinações geradas em 'col_v3'.
DADOS

['sigla_uf', 'sexo', 'raca_cor']
['sigla_uf', 'sexo', 'faixa_etaria']
['sigla_uf', 'sexo', 'grau_instrucao_apos_2005']
['sigla_uf', 'sexo', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'raca_cor', 'faixa_etaria']
['sigla_uf', 'raca_cor', 'grau_instrucao_apos_2005']
['sigla_uf', 'raca_cor', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sigla_uf', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sexo', 'raca_cor', 'faixa_etaria']
['sexo', 'raca_cor', 'grau_instrucao_apos_2005']
['sexo', 'raca_cor', 'cnae_2_subclasse_descricao_secao']
['sexo', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sexo', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sexo', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005']
['raca_cor', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['raca_cor', 'grau

,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
26942,NaN,NaN,NaN,Faixa etária desconhecida,SUP. INCOMP,Informação E Comunicação,62
26943,NaN,NaN,NaN,Faixa etária desconhecida,SUP. INCOMP,Outras Atividades De Serviços,931
26944,NaN,NaN,NaN,Faixa etária desconhecida,SUP. INCOMP,Saúde Humana E Serviços Sociais,208
26945,NaN,NaN,NaN,Faixa etária desconhecida,SUP. INCOMP,"Transporte, Armazenagem E Correio",54


In [ ]:
# Processamento de Combinações e Concatenação de Dados (col_v4)

# Gera uma lista de combinações para agrupamento de dados.
# O argumento '4' indica o nível ou tipo de combinações a serem geradas.
col_v4 = lista_de_combinacoes(4)

# Itera sobre cada combinação na lista 'col_v4'.
for iter4 in col_v4:
    # Chama a função 'contar_por_grupo' para processar os dados
    # utilizando a conexão 'con_rais', a fonte 'rais2023' e a combinação atual 'iter4'.
    dados_v4 = contar_por_grupo(con_rais,'rais2023',iter4)

    # Concatena os resultados 'dados_v4' ao DataFrame principal 'DADOS'.
    # `pd.concat` adiciona as novas linhas ao final de 'DADOS'.
    # `ignore_index=True` redefine o índice do DataFrame resultante para ser contínuo.
    DADOS = pd.concat([DADOS,dados_v4],ignore_index=True)

    # Imprime a combinação atual ('iter4') para acompanhar o progresso do loop.
    print(iter4)

# Exibe o DataFrame 'DADOS' final, que agora inclui os resultados de todas as contagens
# para as combinações geradas em 'col_v4'.
DADOS

['sigla_uf', 'sexo', 'raca_cor', 'faixa_etaria']
['sigla_uf', 'sexo', 'raca_cor', 'grau_instrucao_apos_2005']
['sigla_uf', 'sexo', 'raca_cor', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'sexo', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sigla_uf', 'sexo', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'sexo', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sigla_uf', 'raca_cor', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'raca_cor', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'faixa_etaria', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sexo', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sexo', 'raca_cor', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sexo', 'raca_cor', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sexo', 'faixa_etaria', 'grau_instrucao_apos_2

,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
156785,NaN,NaN,Preta,Faixa etária desconhecida,SUP. INCOMP,Indústrias De Transformação,4
156786,NaN,NaN,Preta,Faixa etária desconhecida,SUP. INCOMP,Informação E Comunicação,1
156787,NaN,NaN,Preta,Faixa etária desconhecida,SUP. INCOMP,Outras Atividades De Serviços,60
156788,NaN,NaN,Preta,Faixa etária desconhecida,SUP. INCOMP,Saúde Humana E Serviços Sociais,2


In [ ]:
# Processamento de Combinações e Concatenação de Dados (col_v5)

# Gera uma lista de combinações para agrupamento de dados.
# O argumento '5' provavelmente indica um nível ou tipo específico de combinações.
col_v5 = lista_de_combinacoes(5)

# Itera sobre cada combinação na lista 'col_v5'.
for iter5 in col_v5:
    # Chama a função 'contar_por_grupo' para processar os dados
    # usando a conexão 'con_rais', a fonte 'rais2023' e a combinação atual 'iter5'.
    dados_v5 = contar_por_grupo(con_rais,'rais2023',iter5)

    # Concatena os resultados 'dados_v5' ao DataFrame principal 'DADOS'.
    # 'pd.concat' adiciona as novas linhas ao final de 'DADOS'.
    # 'ignore_index=True' redefine o índice do DataFrame resultante, garantindo que seja contínuo.
    DADOS = pd.concat([DADOS,dados_v5],ignore_index=True)

    # Imprime a combinação atual ('iter5') para acompanhar o progresso do loop.
    print(iter5)

# Exibe o DataFrame 'DADOS' final, que agora contém os resultados de todas as contagens
# para as combinações geradas em 'col_v5'.
DADOS

['sigla_uf', 'sexo', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005']
['sigla_uf', 'sexo', 'raca_cor', 'faixa_etaria', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'sexo', 'raca_cor', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'sexo', 'faixa_etaria', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sigla_uf', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']
['sexo', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']


,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
430146,NaN,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,Indústrias De Transformação,2
430147,NaN,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,Informação E Comunicação,1
430148,NaN,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,Outras Atividades De Serviços,55
430149,NaN,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,Saúde Humana E Serviços Sociais,1


In [ ]:
# Gera uma lista de combinações para agrupamento de dados.
col_v6 = lista_de_combinacoes(6)

# Itera sobre cada combinação na lista 'col_v6'.
for iter6 in col_v6:
    # Chama a função 'contar_por_grupo' para processar os dados com base na combinação atual.
    dados_v6 = contar_por_grupo(con_rais,'rais2023',iter6)

    # Concatena os resultados 'dados_v6' ao DataFrame principal 'DADOS'.
    # 'ignore_index=True' redefine o índice do DataFrame resultante.
    DADOS = pd.concat([DADOS,dados_v6],ignore_index=True)

    # Imprime a combinação atual para acompanhamento do progresso.
    print(iter6)

# Exibe o DataFrame 'DADOS' final após todas as concatenações.
DADOS

['sigla_uf', 'sexo', 'raca_cor', 'faixa_etaria', 'grau_instrucao_apos_2005', 'cnae_2_subclasse_descricao_secao']


,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,NaN,NaN,NaN,NaN,NaN,241574
2,AL,NaN,NaN,NaN,NaN,NaN,858262
3,AM,NaN,NaN,NaN,NaN,NaN,1078184
4,AP,NaN,NaN,NaN,NaN,NaN,208879
...,...,...,...,...,...,...,...
618030,TO,Masculino,Preta,Faixa etária desconhecida,MEDIO COMPL,Outras Atividades De Serviços,24
618031,TO,Masculino,Preta,Faixa etária desconhecida,SUP. COMP,"Administração Pública, Defesa E Seguridade Social",39
618032,TO,Masculino,Preta,Faixa etária desconhecida,SUP. COMP,Eletricidade E Gás,1
618033,TO,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,"Administração Pública, Defesa E Seguridade Social",3


In [ ]:
DADOS.to_csv("estoque_rais2023_v2.csv",sep = ';', index= False)

## Realizando Merge

In [ ]:
# Vetores
local = ["", "CE", "PE", "BA"]
sexo = ["", "Feminino", "Masculino", "Não informado"]
raca_cor = ["", "Branca", "Preta", "Parda", "Amarela", "Indígena",
            "Não informada", "Não identificada"]
grau_de_instrucao = ["", "ANALFABETO", "ATE 5.A INC", "5.A CO FUND", "6.A.9.FUND",
                     "FUND COMPL", "MEDIO INCOMP", "MEDIO COMPL",
                     "SUP.INCOMP", "SUP.COMP", "MESTRADO", "DOUTORADO"]
setores = ["",
           "Agricultura, Pecuária, Produção Florestal, Pesca e Aquicultura",
           "Indústrias Extrativas",
           "Indústrias de Transformação",
           "Eletricidade e Gás",
           "Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação",
           "Construção",
           "Comércio; Reparação de Veículos Automotores e Motocicletas",
           "Transporte, Armazenagem e Correio",
           "Alojamento e Alimentação",
           "Informação e Comunicação",
           "Atividades Financeiras, de Seguros e Serviços Relacionados",
           "Atividades Imobiliárias",
           "Atividades Profissionais, Científicas e Técnicas",
           "Atividades Administrativas e Serviços Complementares",
           "Administração Pública, Defesa e Seguridade Social",
           "Educação",
           "Saúde Humana e Serviços Sociais",
           "Artes, Cultura, Esporte e Recreação",
           "Outras Atividades de Serviços",
           "Serviços Domésticos",
           "Organismos Internacionais e Outras Instituições Extraterritoriais",
           "Não identificado"]
idades = ["", "14 a 17", "18 a 24", "25 a 39", "40 a 59", "60 ou mais"]

In [ ]:
# inputando valores em panco em nulos
DADOS = DADOS.fillna("")
DADOS

,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,AC,,,,,,241574
2,AL,,,,,,858262
3,AM,,,,,,1078184
4,AP,,,,,,208879
...,...,...,...,...,...,...,...
618030,TO,Masculino,Preta,Faixa etária desconhecida,MEDIO COMPL,Outras Atividades De Serviços,24
618031,TO,Masculino,Preta,Faixa etária desconhecida,SUP. COMP,"Administração Pública, Defesa E Seguridade Social",39
618032,TO,Masculino,Preta,Faixa etária desconhecida,SUP. COMP,Eletricidade E Gás,1
618033,TO,Masculino,Preta,Faixa etária desconhecida,SUP. INCOMP,"Administração Pública, Defesa E Seguridade Social",3


In [ ]:
# Filtragem
dados_alterados = DADOS.query("sigla_uf in @local and faixa_etaria in @idades")

In [ ]:
# Salvando dados filtrados

# Exporta o DataFrame 'dados_alterados' para um arquivo CSV.
dados_alterados.to_csv(
    "resultado_final_estoque_rais2023.csv", # Nome do arquivo de saída.
    sep = ";",                             # Define o ponto e vírgula como separador de campos.
    index = False                          # Evita que o índice do DataFrame seja salvo como uma coluna.
)

In [ ]:
dados_filtrados = pd.read_csv("/content/resultado_final_estoque_rais2023.csv",sep = ';')
dados_filtrados = dados_filtrados.fillna("")
dados_filtrados

In [5]:
# Padronizar todos os campos relevantes no DataFrame de dados
dados_filtrados['sigla_uf'] = dados_filtrados['sigla_uf'].fillna('').str.strip().str.upper()
dados_filtrados['sexo'] = dados_filtrados['sexo'].fillna('').str.strip().str.upper()
dados_filtrados['raca_cor'] = dados_filtrados['raca_cor'].fillna('').str.strip().str.upper()
dados_filtrados['faixa_etaria'] = dados_filtrados['faixa_etaria'].fillna('').str.strip().str.upper()
dados_filtrados['grau_instrucao_apos_2005'] = dados_filtrados['grau_instrucao_apos_2005'].fillna('').str.strip().str.upper()
dados_filtrados['cnae_2_subclasse_descricao_secao'] = dados_filtrados['cnae_2_subclasse_descricao_secao'].fillna('').str.strip().str.upper()

In [6]:
dados_filtrados

,sigla_uf,sexo,raca_cor,faixa_etaria,grau_instrucao_apos_2005,cnae_2_subclasse_descricao_secao,estoque
0,,,,,,,81537467
1,BA,,,,,,3797921
2,CE,,,,,,2470312
3,PE,,,,,,2652572
4,,FEMININO,,,,,35939355
...,...,...,...,...,...,...,...
94277,PE,MASCULINO,PRETA,60 OU MAIS,SUP. INCOMP,ELETRICIDADE E GÁS,1
94278,PE,MASCULINO,PRETA,60 OU MAIS,SUP. INCOMP,INDÚSTRIAS DE TRANSFORMAÇÃO,2
94279,PE,MASCULINO,PRETA,60 OU MAIS,SUP. INCOMP,INFORMAÇÃO E COMUNICAÇÃO,3
94280,PE,MASCULINO,PRETA,60 OU MAIS,SUP. INCOMP,OUTRAS ATIVIDADES DE SERVIÇOS,1


In [8]:
def filtro(dados,local_,sexo_,raca_,idade_,gdi_,setor_,coluna):
    # Função para filtrar um DataFrame 'dados' com base em múltiplos critérios
    # e retornar o valor de uma 'coluna' específica.

    # Parâmetros:
    # dados: DataFrame a ser filtrado.
    # local_, sexo_, raca_, idade_, gdi_, setor_: Critérios de filtragem para as respectivas colunas.
    # coluna: Nome da coluna cujo valor será retornado.

    # Realiza a filtragem do DataFrame 'dados' usando todas as condições fornecidas.
    # Seleciona a 'coluna' das linhas que satisfazem todos os critérios.
    contagem = dados[(dados['sigla_uf'] == local_) &
                                 (dados['sexo'] == sexo_) &
                                 (dados['raca_cor'] == raca_) &
                                 (dados['faixa_etaria'] == idade_) &
                                 (dados['grau_instrucao_apos_2005'] == gdi_) &
                                 (dados['cnae_2_subclasse_descricao_secao'] == setor_)
                                ][coluna]

    # Retorna o primeiro valor encontrado na série 'contagem' se ela não estiver vazia.
    # Caso 'contagem' esteja vazia (nenhuma linha correspondeu aos filtros), retorna 0.
    return contagem.iloc[0] if len(contagem) > 0 else 0

In [7]:
# Listas de categorias para filtragem e combinação de dados.
# O elemento vazio no início de cada lista serve como um valor "coringa" ou "todos".

# Unidades Federativas (UF) a serem consideradas.
local = ["", "CE", "PE", "BA"]

# Opções de gênero.
sexo = ["", "Feminino", "Masculino", "Não informado"]

# Categorias de raça/cor.
raca_cor = ["", "Branca", "Preta", "Parda", "Amarela", "Indígena","Não informada", "Não identificado"]

# Níveis de instrução (escolaridade).
grau_de_instrucao = ["", "ANALFABETO", "ATE 5.A INC", "5.A CO FUND", "6. A 9. FUND",
"FUND COMPL", "MEDIO INCOMP", "MEDIO COMPL",'SUP. INCOMP', "SUP. COMP", "MESTRADO", "DOUTORADO"]

# Setores de atividade econômica.
setores = ["","Agricultura, Pecuária, Produção Florestal, Pesca E Aquicultura",
           "Indústrias Extrativas","Indústrias De Transformação","Eletricidade E Gás",
           "Água, Esgoto, Atividades De Gestão De Resíduos E Descontaminação","Construção",
           "Comércio; Reparação De Veículos Automotores E Motocicletas",
           "Transporte, Armazenagem E Correio","Alojamento E Alimentação",
           "Informação E Comunicação","Atividades Financeiras, De Seguros E Serviços Relacionados",
           "Atividades Imobiliárias","Atividades Profissionais, Científicas E Técnicas",
           "Atividades Administrativas E Serviços Complementares",
            "Administração Pública, Defesa E Seguridade Social",
            "Educação",
            "Saúde Humana E Serviços Sociais",
            "Artes, Cultura, Esporte E Recreação",
            "Outras Atividades De Serviços",
            "Serviços Domésticos",
            "Organismos Internacionais E Outras Instituições Extraterritoriais",
            "Não identificado"]

# Faixas etárias.
idades = ["", "14 a 17", "18 a 24", "25 a 39", "40 a 59", "60 ou mais"]

## Geração de Combinações e DataFrame

# Lista para armazenar dicionários de todas as combinações de critérios.
combinacoes = []

# Loops aninhados para gerar todas as combinações possíveis entre os vetores definidos.
for uf in local:
    for s in sexo:
      for r in raca_cor:
        for idade in idades:
          for g in grau_de_instrucao:
            for setor in setores:
                # Adiciona um dicionário à lista 'combinacoes' para cada combinação.
                # A função 'filtro' é chamada para obter o valor de 'estoque'
                # correspondente à combinação atual.
                   combinacoes.append({
                    'uf': uf,
                    'sexo': s,
                    'raca': r,
                    'idade': idade,
                    'grau_de_instrucao': g,
                    'secao': setor,
                    'estoque': filtro(
                        dados_filtrados,
                        uf.strip().upper(),
                        s.strip().upper(),
                        r.strip().upper(),
                        idade.strip().upper(),
                        g.strip().upper(),
                        setor.strip().upper(),
                        'estoque')})

                   print(uf,s,r,idade,g,setor)

# Converte a lista de dicionários 'combinacoes' em um DataFrame do pandas.
df = pd.DataFrame(combinacoes)
# Exibe o DataFrame resultante.
df

NameError: name 'filtro' is not defined

In [ ]:
df.to_csv("resultado_rais2023.csv",sep = ';',index = False)

In [ ]:
from google.colab import files
files.download('resultado_rais2023.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Recarregar os arquivos com o separador correto ";"
df_final = pd.read_csv("/content/resultado_final_estoque_rais2023.csv", sep=";")
df_combinacoes = pd.read_csv("/content/resultado_rais2023.csv", sep=";")

# Verificar a quantidade de registros com estoque > 0 em cada um
quant_final_estoque_positivo = (df_final["estoque"] > 0).sum()
quant_combinacoes_estoque_positivo = (df_combinacoes["estoque"] > 0).sum()

quant_final_estoque_positivo, quant_combinacoes_estoque_positivo


(np.int64(94282), np.int64(93479))